##### Get parameters from config .yaml file

In [1]:
import pathlib
from cfdmod.use_cases.block_gen import *

cfg_file = pathlib.Path("./fixtures/tests/block_gen/block_params.yaml")
cfg = GenerationParams.from_file(cfg_file)

cfg

GenerationParams(N_blocks_x=10, N_blocks_y=10, block_params=BlockParams(height=10.0, width=10.0, length=10.0), spacing_params=SpacingParams(spacing_x=10.0, spacing_y=10.0, line_offset=10.0, is_abs=True, offset_direction=<OffsetDirection.x: 'x'>))

##### Calculate spacings and offset from parameters

In [ ]:
single_line_repeat = (
    cfg.N_blocks_y - 1
    if cfg.spacing_params.offset_direction == "y"
    else cfg.N_blocks_x - 1
)
single_line_spacing = (
    cfg.spacing_params.spacing_x + cfg.block_params.length
    if cfg.spacing_params.offset_direction == "x"
    else cfg.spacing_params.spacing_y + cfg.block_params.width
)
multiple_line_repeat = (
    cfg.N_blocks_x - 1
    if cfg.spacing_params.offset_direction == "y"
    else cfg.N_blocks_y - 1
)
multiple_line_spacing = (
    cfg.spacing_params.spacing_x + cfg.block_params.length
    if cfg.spacing_params.offset_direction == "y"
    else cfg.spacing_params.spacing_y + cfg.block_params.width
)

In [ ]:
from cfdmod.api.geometry.STL import export_stl
import pathlib
from cfdmod.use_cases.block_gen import *


vertices, triangles = build_single_block(cfg.block_params)



single_line_vertices, single_line_triangles = linear_pattern(
    vertices,
    triangles,
    direction=cfg.spacing_params.offset_direction,
    n_repeats=single_line_repeat,
    spacing_value=single_line_spacing,
)

full_vertices, full_triangles = linear_pattern(
    single_line_vertices,
    single_line_triangles,
    direction=cfg.perpendicular_direction,
    n_repeats=multiple_line_repeat,
    spacing_value=multiple_line_spacing,
    offset_value=cfg.calculate_spacing(direction=cfg.perpendicular_direction),
)

export_stl(output_path / "blocks.stl", full_vertices, full_triangles)